In [5]:
#Imports
import pandas as pd
import os.path
import random
import time
import string
import webbrowser
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import itertools

def text_between_subtrings(s, start, end):
    return s.find(start), s.find(end)

def wordify(question, max_words=10):
    question_words=question.split()
    if len(question_words)>max_words:
        question_words=question_words[:max_words]
    q=' '.join(question_words)
    #q=''.join("0" if c.isdigit() else c for c in q)
    q=re.sub('\d+', '#', q)#convert multiple digits to single digits
    q=re.sub('#.#', '#', q)#convert multiple digits to single digits
    q=re.sub('_+', '', q)
    q=re.sub('\d+\)', '', q)#these two lines get rid of any 3) _______ crap
    if "version is this" in q:
        q = re.sub('[\bA-E]', 'V', q)  
        q = re.sub('#\)', '', q)
    else:
        "Make sure the version question reads: What version is this test"
    return q.strip()

def extract_questions(s, max_words=10):
    more='yes'
    n=1
    questionlist=[]
    while more=='yes':
        start = '\n'+str(n)
        end = '\n'+str(n+1)
        p1, p2 = text_between_subtrings(s, start, end)
        if p1!=-1 and p2!=-1:
            txt=s[p1+len(start):p2]
            ###Grab the part between ) and nA, and convert into words
            pp1, pp2 = text_between_subtrings(txt, ')', '\nA')
            txt=txt[1:txt.find('\nA')]
            txt=wordify(txt,max_words)
            questionlist.append(txt)
            n+=1
        elif p1!=-1 and p2==-1:
            txt=s[p1+len(start):]
            pp1, pp2 = text_between_subtrings(txt, ')', '\nA')
            txt=txt[1:txt.find('\nA')]
            txt=wordify(txt,max_words)
            #re.sub('[^A-Z]', '', s)
            questionlist.append(txt)
            more='no'
        else:
            print("Sum ting wong. Shouldn't go here. p1= "+str(p1)+" p2= "+str(p2))
            more='no'
    return questionlist
    

def check_uniqueness_of_QIDs(All_IDs):
    rr=range(len(All_IDs))
    combs=list(itertools.combinations(rr,2))
    for comb in combs:
        u=[x for x in All_IDs[comb[0]] if x not in All_IDs[comb[1]]]
        if u:
            print("Oops! Some question ID's are not unique: "+str(comb[0])+' and '+str(comb[1]))

def createQuestionIDs(max_words=10):
    All_IDs=[]
    for n in range(0,6):
        filename="key"+str(n)+".txt"
        if (os.path.isfile(filename)):
            with open(filename, encoding="utf-8") as file:  
                data = file.read()
            p=data.rfind('\n\n\n\n1)')
            if p!=-1: 
                questions=data[:p]
            else:
                print('Make sure questions and answers in key'+str(n)+' are separated by \n\n\n\n')
                return []
            All_IDs.append(extract_questions(questions, max_words))
            check_uniqueness_of_QIDs(All_IDs)
    return All_IDs      

def getAllKeys(fixed_points_per_question=1):
    '''read all files named key0-key4, make them into strings and and return a dictionary
    containing a list of keys and also the number of questions'''
    keylist=[]
    pointlist=[]
    for n in range(0,6):
        filename="key"+str(n)+".txt"
        if (os.path.isfile(filename)):
            with open(filename, encoding="utf-8") as file: 
                data = file.read()
            u=re.search('\n\n\n\n+1\)',data)
            if u is not None:
                answers=data[u.start():].strip()
            else:
                print('Make sure questions and answers in key'+str(n)+' are separated by \n\n\n\n')
                return None

            ukey=re.findall('[\bA-E]',answers)
            ukey=[str(ord(x)-65) for x in ukey]
            key=''.join(ukey)
            keylist.append(key)
            
            qnum=len(key)

            #This assumes that the answerkey contains points EITHER in the user-supplied form 8) A /tab 3/n OR testgen-supplied form "Points: 3"
            upoints=re.findall('\t\d+',answers)+re.findall(':\s\d+',answers)
            if upoints:
                _=''.join(upoints)
                points=re.sub('[^ \d+]',' ', _).split()
                points=[int(x) for x in points ]
                assert(len(points)==qnum)
            else:
                points=[fixed_points_per_question]*qnum
            
            points[-1]=0
            pointlist.append(points)
    
    if not keylist:
        print("Answerkeys not found. Make sure they are named key0.txt, key1.txt etc")
    return {"keylist":keylist,"pointlist":pointlist,"numberOfQuestions":qnum}

def process_grades(data,outs, QIDs,analysis=False):
    '''grades all exams using correct keys, writes questions missed and scores'''

    for NN in range(0, data.shape[0]):
        #print("NN: "+str(NN))
        ans=data.iat[NN,2]
        if len(ans)==numberOfQuestions-1:
            v=guess_the_version(ans)
            ans=ans+str(v)
            print("Assuming version "+invkeydictionary[v]+ " for: "+data.iat[NN,1]+" (Srl No: "+data.iat[NN,0]+").")
        check1=gradeWithKeylist(ans, outs, QIDs, analysis, NN)
        data.iat[NN,3]=check1['missed']
        data.iat[NN,4]=check1['score']
        data.iat[NN,5]=100.0*float(check1['score'])/float(new_totalpoints)

    if analysis:
        #remove the column with the version numbers
        analysis_df.drop(analysis_df.columns[analysis_df.shape[1]-2], axis=1, inplace=True)
    return data


def gradeWithKeylist(ans,outs, QIDs, analysis=False, N=0):
    '''multiple versions - find the correct key as indicated on the last question on the exam '''
    keylist=outs["keylist"]
    pointlist=outs["pointlist"]
    numberOfQuestions=outs["numberOfQuestions"] 
    
    assert(keylist!=[])
    assert (ans[-1:] in ['0','1','2','3','4'])
    whichKey=int(ans[-1:])
    
    key=keylist[whichKey]
    points=pointlist[whichKey]
    assert len(key)==len(ans)
    missed=""
    rejalt=[1]*numberOfQuestions
    for n in range(0,len(key)-1):
        if key[n]!=ans[n]:
            #print("storing")
            missed+=str(n+1)+", "
            rejalt[n]=0
    if sum(rejalt)==numberOfQuestions:
        missed="ALL CORRECT"
    else:
        missed="v"+invkeydictionary[whichKey]+": "+missed[:len(missed)-2]
    score=sum([i*j for i,j in zip(points,rejalt)])
    

    mydict1 = dict(zip(QIDs[whichKey],rejalt))
    #sortedIDs=sorted(mydict1.keys())
    sorted_rejalt=[mydict1[k] for k in QIDs[0]]####its sorted according to v0
    

    
    if analysis:   
        sorted_rejalt.append(score)
        analysis_df.loc[N] = sorted_rejalt 
        
        mydict2 = dict(zip(QIDs[whichKey],ans))  
        #sortedIDs=sorted(mydict2.keys())
        sorted_ans=[mydict2[k] for k in QIDs[0]]
        allAnswers_df.loc[N] = sorted_ans
    

        
    return {'missed':missed, 'score':score}


def guess_the_version(no_version):
    scores=[]
    for n in range(numberOfVersions):
        try_version=no_version+str(n)
        check1=gradeWithKeylist(try_version, outs, QIDs, analysis=False)
        scores.append(check1['score'])
    return scores.index(max(scores))

def count_how_many(col, value):
    '''takes a pandas series "col" and counts the number of instances of a "value" 
    Not used since pandas has a built-in function'''
    
    count=0
    for n in range(0,col.shape[0]):
        if col[n]==value:
            count+=1
    return count


def addStarsToCorrectChoices(rdf, keylist, m, QIDS):
    '''adds stars to correct choices in rdf'''
    for n in range(0,rdf.shape[0]):
        if m!=0:
            mydict0 = dict(zip(QIDs[m],keylist[m])) 
            sortedkey=[mydict0[x] for x in QIDs[0]]
        else:
            sortedkey=keylist[m]
        the_correct_answer=sortedkey[n]
        if the_correct_answer in list(rdf):
            rdf.iloc[n][the_correct_answer]=rdf.iloc[n][the_correct_answer]+"*"
    return rdf

def analyse_items(a_df, QIDs):
    '''create separate dfs with how many marked correct for each version separately'''
    vers=['0','1','2','3','4']
    lvers=['A','B','C','D','E']
    outvars=[]
    versionsfound=[]
    for n in range(0,len(QIDs)):
        ch=str(n)
        ##check the last column "which version..." to find the version and see if it matches ch
        part_df = a_df.loc[a_df[QIDs[0][-1]]==ch]
        if not part_df.empty:
            print("Analysing data for version: "+lvers[n])
            part_df=part_df.T
            part_df=part_df.apply(pd.Series.value_counts, axis=1).fillna(0)
            part_df=part_df.applymap(int)
            part_df=part_df.applymap(str)
            part_df=addStarsToCorrectChoices(part_df,keylist,n, QIDs)
            if ' ' in list(part_df):
                part_df.rename(columns={' ': 'blank'}, inplace=True)   
            for pp in vers:
                if pp in list(part_df):
                    part_df.rename(columns={pp: lvers[int(pp)]}, inplace=True)
            outvars.append(part_df)
            versionsfound.append(lvers[n])
    
#     Return a dict so that the html function knows which version each table belongs to    
    return dict(zip(versionsfound,outvars))

def make_item_analysis(a_df):
    
    item_analysis_df=pd.DataFrame(index=["Difficulty","Discrimination"],columns = list(analysis_df))
    n=a_df.shape[0]
    v=analysis_df.values#converts into np array
    
    
    #Calculating Difficulty
    u1=np.sum(v,axis=0)/n
    item_analysis_df.loc["Difficulty"]=u1[0:u1.shape[0]]
    
    #Calculating Discrimination
    students_per_group=n//3
    if students_per_group<2:
        print("Need more students for discrimination analysis (at least 7).")
    else:
        v=v[0:v.shape[0]-1,:]#exclude the bottom row which had the sums (total number correct for each q)
        v=v[v[:,v.shape[1]-1].argsort()[::-1]]#sort descending by scores
        #print(v)
        u1=np.sum(v[0:students_per_group,:],axis=0)#top scorers
        u2=np.sum(v[n-students_per_group : n,:],axis=0)#bottom scorers
        disc=(u1-u2)/students_per_group
        disc.shape
        item_analysis_df.loc["Discrimination"]=disc[0:disc.shape[0]]
    
    item_analysis_df.drop("score", axis=1, inplace=True)
    return item_analysis_df.T

#################Writing data#############################
def write_to_xl(adf, rfilename):
    adf.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})#This facilitates the Vlookup later
    writer = pd.ExcelWriter(rfilename+'_processed.xlsx')
    adf.to_excel(writer,'Sheet1',index=False)
    writer.save()
    
def write_to_csv(adf, rfilename):
    filename = rfilename+'_processed.csv'
    #adf=adf.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})#This facilitates the Vlookup later
    adf.to_csv(filename, index=False)
    

def write_to_webpage(j_df, a_df,ia_df,ncbv={}, histobin=11):
    filename = rawdatafilename+'_summary.html'
    f = open(filename,'w',encoding='utf8')
    
    uu=df['Percentage'].describe().to_frame()
    uu.rename(columns={'Percentage': "Value"}, index={'count':'Number of Students','mean':'Mean(%)', 'std':'Standard Deviation', 'min':'Lowest(%)','25%':'25th percentile', '50%':'50th percentile', '75%':'75th percentile', 'max':"Highest(%)"},inplace=True)
    uu.loc['Maximum Available(%) '] = [100*totalpoints/new_totalpoints]
    uu.loc['Points dropped '] = [point_drop]

    pre="<h2>Summary Data:</h2>"
    summary_table=uu.to_html(float_format=lambda x: '%10.2f' % x).replace("dataframe" ,"sumdat")
    summary_table=pre+summary_table
    post="<p>Score Distribution:</p>"
    
    figure = plt.figure()
    df['Percentage'].plot(kind='hist', bins=10)
    plt.xlim(xmax=100*totalpoints/new_totalpoints)
    figure.savefig('histo.svg')
    
    pre="<h2>Score Distribution:</h2>"
    image_code='<img src="histo.svg" alt="histogram">'
    image=pre+image_code

    pre="<h2>Item Analysis:</h2>"
    item_analysis_table=ia_df.sort_values('Difficulty').to_html(float_format=lambda x: '%10.2f' % x).replace("dataframe" ,"customers")
    item_analysis_table=pre+item_analysis_table

    missed_tables=""
    if ncbv:
        for ver in ['A', 'B','C','D','E']:
            if ver in ncbv.keys():
                pre="<h2>Distractor Analysis for Version "+ver+":</h2>"
                post=ncbv[ver].to_html().replace("dataframe" ,"howmany")
                missed_tables+=pre+post


    html_start = """<html>
    <head>
     <link rel="stylesheet" type="text/css" href="mystyle.css">
    </head>
    <body>
    <h1>Data Analysis</h1>
    """


    html_end='''</body>
    </html>'''
    message=html_start+summary_table+image+item_analysis_table+missed_tables+html_end
    f.write(message)
    f.close()

    #Change path to reflect file location
    webbrowser.open_new_tab(filename)

###############################################
## Functions that create fake data#############

def create_afake(number_of_versions, blanks=False):
    '''creates a fake answerkey'''
    ansstring=[]
    if not blanks:
        choices=[0,1,2,3,4]
    else:
        choices=[0,1,2,3,4,' ']
    for n in range(25):
        ans=random.randrange(0,len(choices))
        ansstring.append(str(choices[ans]))
    ans=random.randrange(0,number_of_versions)
    ansstring.append(str(ans))
    ansstring=''.join(ansstring)
    return ansstring
def create_fakes(N, number_of_versions, blanks=False):
    '''creates fakes and writes to an excel file'''
    fakes_df=pd.DataFrame(index=range(N),columns = range(3),dtype=str)
    fakes_df.iloc[:,0:2] =""
    for i in range(0,N):
        fakes_df.iat[i,2]=create_afake(number_of_versions, blanks)
    writer = pd.ExcelWriter('fakes.xlsx')
    fakes_df.to_excel(writer,'Sheet1')
    writer.save()

########################################################
############### Checking for incorrect Srl Nos################

def find_oks(row):
    return row['Name'].find(row['First Name'].upper())!=-1 and not row['Name'].isnull()
def remove_ok_rows(df):
    notnull=df.dropna()
    okrows=notnull.loc[notnull.apply(find_oks, axis=1)]
    oksremoved=pd.concat([df, okrows]).drop_duplicates(keep=False)
    return oksremoved

def check_serial_numbers():
    df=pd.read_csv('BSGrades.csv', usecols=["Last Name", "First Name", "Serial Number Text Grade <Text>"])
    df_all=pd.read_excel('All.xlsx', header=None, parse_cols=1,names = ["Serial Number Text Grade <Text>", "Name"])

    checker_left = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='left')
    checker_right = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='right')
    u1=checker_left[checker_left.isnull().any(axis=1)]
    u2=checker_left[checker_left['Serial Number Text Grade <Text>'].duplicated(keep=False)]
    u3=checker_right[checker_right.isnull().any(axis=1)]
    error_rpt=pd.concat([u1, u2, u3], axis=0)
#     display(error_rpt)
#     if not error_rpt.empty:
#         error_rpt=remove_ok_rows(error_rpt)
    error_rpt["Last Name"] = error_rpt["Last Name"]+' '+error_rpt['First Name']
    error_rpt.drop('First Name', axis=1,inplace=True)
    error_rpt.rename(columns={'Serial Number Text Grade <Text>': 'Srl No', 'Last Name':'Registered Name', 'Name':'Entered Name'}, inplace=True) 
    return error_rpt

Manually convert the dat file into an excel file using excel. Only extract the serial number,
name and Answers, and make sure Answers is text.

In [6]:
#COLLECTING DATA
rawdatafilename='All'
xls_file = pd.ExcelFile(rawdatafilename+'.xlsx')
df = xls_file.parse('Sheet1', header=None, parse_cols=2,names = ["Srl No", "Name", "Answers"], dtype='str')
#parse_cols makes sure that only cols 0,1 and 2 are extracted
#checking for blanks, print only if blanks found
if not df[df['Answers'].str.contains(" ")].empty:
    blankers=df[df['Answers'].str.contains(" ")]
    display(blankers)
ER=check_serial_numbers()
if not ER.empty:
    display(ER)
df["Missed"] = ""
df["Score"]=0
df["Percentage"]=np.nan

,Srl No,Name,Answers
2,1009,HAN KARIN,0110241 21 4040


,Registered Name,Srl No,Entered Name
1,Bolling Atiyaa,1003,NaN
12,Muezzinoglu Mine,1018,NaN
2,Carlson Brynn,1002,CARLSON BRYNN L
3,Carlson Brynn,1002,MUEZZINOGLU MINE


In [3]:
#initializing everything
keydictionary={"A":"0","B":"1","C":"2","D":"3","E":"4"}
invkeydictionary={0:"A",1:"B",2:"C",3:"D",4:"E"}
points_per_question=3
point_drop=2
outs = getAllKeys(points_per_question)
keylist=outs["keylist"]
pointlist=outs["pointlist"]
numberOfQuestions=outs["numberOfQuestions"]
if not keylist or not pointlist or numberOfQuestions==0:
    sys.exit("Keys not properly imported")
totalpoints=sum(pointlist[2])
new_totalpoints=totalpoints-point_drop
numberOfVersions=len(keylist)
numberOfStudents=df.shape[0]
QIDs=createQuestionIDs(numberOfQuestions)
headings=list(QIDs[0])#####headings=QIDs[0] assigns by reference, so changing headings will change QIDs[0]
allAnswers_df = pd.DataFrame(index=range(numberOfStudents), columns = headings)# stores all student answers for each question, questions are the columns
#results_df = pd.DataFrame(index=headings, columns = ["A","B","C","D","E","Skipped","Diff","Disc"])
headings.append("score")
analysis_df  = pd.DataFrame(index=range(numberOfStudents),columns = headings)# stores whether answer was correct, questions are the columns, last column is the score


In [4]:
##Everything initialized. Running the code now
didnt_enter_versions=df[df['Answers'].map(len)!=numberOfQuestions]
if not didnt_enter_versions.empty:
    didnt_enter_versions=didnt_enter_versions[['Srl No', 'Name','Answers']]
    display(didnt_enter_versions)
print("Starting calculations...")
starttime = time.time()
df=process_grades(df,outs, QIDs, analysis=True)
number_correct_by_version=analyse_items(allAnswers_df, QIDs)
item_analysis_df = make_item_analysis(analysis_df) 
endtime = time.time()
print("Done. That took: "+str(endtime-starttime)+ " sec for "+str(numberOfStudents)+" students")
#check for duplicated serial numbers
if not df[df['Srl No'].duplicated(keep=False)].empty:
    print('Warning: Duplicates in serial numbers found!')
    dupes=df[df['Srl No'].duplicated(keep=False)]
max_score=analysis_df.at[analysis_df['score'].idxmax(),'score']

Starting calculations...
Analysing data for version: A
Analysing data for version: B
Analysing data for version: C
Analysing data for version: D
Done. That took: 0.13663434982299805 sec for 20 students


In [ ]:
#write_to_csv(df,rawdatafilename)
write_to_webpage(df, analysis_df, item_analysis_df,number_correct_by_version)

In [6]:
#Grades and displays the N'th entry in the list using a keylist
#N=random.randint(1,df.shape[0]-1)
#for N in range(df.shape[0]-1):
N=13
check1=gradeWithKeylist(df.iat[N,2], outs, QIDs, analysis=False)
print("Row "+ str(N)+": "+df.iat[N,1]+", "+str(df.iat[N,0])+". Missed "+str(check1['missed'])+ ". Scored "+str(check1['score'])+"/"+str(totalpoints))
print('That is: '+str(100*check1['score']/new_totalpoints)+'% with '+str(point_drop)+' points dropped')

Row 13: CARLSON BRYNN L, 2009. Missed ALL CORRECT. Scored 42/42
That is: 105.0% with 2 points dropped


In [19]:
####ANSWER EXTRACTION############################

###This part just reads the questions as usual
filename="key"+str(0)+".txt"
if (os.path.isfile(filename)):
    with open(filename, encoding="utf-8") as file:  
        data = file.read()
    p=data.rfind('\n\n\n\n1)')
    if p!=-1: 
        questions=data[:p]

In [20]:
##This parts gets the answer choices for each question and stores them in an array
for n in range(1,16):
#n=15
    s=questions
    txt=''
    start = '\n'+str(n)
    end = '\n'+str(n+1)
    p1, p2 = text_between_subtrings(s, start, end)
    if p1!=-1 and p2!=-1:
        txt=s[p1+len(start):p2]
    if p1!=-1 and p2==-1:
        txt=s[p1+len(start):]
    anslist=[]
    for m in range(0,4):
        start=invkeydictionary[m]+')'
        end=invkeydictionary[m+1]+')'
        #p1, p2 = text_between_subtrings(txt, start, end)
        p1=txt.rfind(start)
        p2=txt.rfind(end)
    #     print('p1= '+str(p1)+', p2= '+str(p2))
        ans=txt[p1:p2].strip()
        if ans:
            anslist.append(ans)
        else:
            anslist.append('')
    if p2!=-1:
        ans=txt[p2:].strip()
        anslist.append(ans)
    else:
        anslist.append('')
    print(anslist)

['A) 4.60 m/s2', 'B) 9.80 m/s2', 'C) 2.30 m/s2', 'D) 3.29 m/s2', 'E) 3.07 m/s2']
['A) 900 m/s.', 'B) 1300 m/s.', 'C) 1200 m/s.', 'D) 1100 m/s.', 'E) 1000 m/s.']
['A) -2.2 N • m.', 'B) 2.2 N • m.', 'C) 4.0 N • m.', 'D) -4.0 N • m.', 'E) zero']
['A) 35 s.', 'B) 50 s.', 'C) 268 s.', 'D) 25 s.', 'E) 13 s.']
['A) 2.4 m/s', 'B) 5.4 m/s', 'C) 4.6 m/s', 'D) None of these', 'E) 3.0 m/s']
['A) 10.0  m/s2', 'B) 13.3 m/s2', 'C) 30.0  m/s2', 'D) 16.0 m/s2', 'E) None of these']
['A) 40.0 N', 'B) 107 N', 'C) 13.3 N', 'D) 26.7 N', 'E) 16.7 N']
['A) 7/5 I', 'B) 2/7 I', 'C) 1/7 I', 'D) 2/5 I', 'E) 3/5 I']
['A) to your right', 'B) to your left', 'C) down', 'D) forwards', 'E) up']
['A) Deflect the ball.', 'B) Catch the ball.', 'C) Your final speed on the skateboard will be the same regardless whether you catch the ball or deflect the ball.', '', '']
['A) sphere, hoop, disk', 'B) hoop, disk, sphere', 'C) sphere, disk, hoop', 'D) hoop, sphere, disk', 'E) disk, hoop, sphere']
['A) (3/2)mb2', 'B) (1/3)mb2', '

In [12]:
txt

') What version is this test? (Please answer C)      15) ______\nA)      B)      C)      D)'

In [155]:
aa.T

AttributeError: 'list' object has no attribute 'T'